In [2]:
import pandas as pd
from collections import Counter
import os

import warnings

warnings.simplefilter("ignore")

In [14]:
df_store = pd.read_csv("./店家名單.csv", encoding="utf-8-sig")
df_store.columns = ["shop_name", "url", "total_score", "food_type"]
df_store.head()

,shop_name,url,total_score,food_type
0,早到晚到 (彰化中正店),/restaurant/mq7f/zao-dao-wan-dao-zhang-hua-zho...,5 stars out of 5,台式
1,火車頭小吃 (彰化店),/restaurant/pui4/huo-che-tou-xiao-chi-zhang-hu...,4.8 stars out of 5,台式
2,可不可熟成紅茶 (彰化中正店),/restaurant/tdxz/ke-bu-ke-shou-cheng-hong-cha-...,4.9 stars out of 5,飲料
3,霖生炒鴨肉羹,/restaurant/g58s/lin-sheng-chao-ya-rou-geng,4.5 stars out of 5,台式
4,茶經鮮奶茶專賣店,/restaurant/nz2a/cha-jing-xian-nai-cha-zhuan-m...,4.9 stars out of 5,飲料


In [7]:
df_food = pd.read_csv("./完整餐點.csv", encoding="utf-8-sig")
df_food.head()

,pic_id,food_name,food_id,introduce,address,longitude,latitude,label,shop_id,sort,price,span
0,mq7f_96743787.jpg,(F)手感豬肉漢堡,96743787,肉品原產地：台灣豬肉,彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,65.0,2
1,mq7f_96743788.jpg,(F)法式牛奶起司鍋燒麵,96743788,NaN,彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,188.0,4
2,mq7f_96743789.jpg,花生厚片,96743789,香濃滑順的花生醬搭配厚片，絲滑入口，等你帶我走！(此描述僅供參考),彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,65.0,2
3,mq7f_96743791.jpg,(F)濃厚乳酪花生法式吐司,96743791,香濃滑順的花生醬搭配吐司，絲滑入口，等你帶我走！(此描述僅供參考),彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,80.0,2
4,mq7f_96743792.jpg,薯泥起司蛋餅,96743792,將綿密薯泥、濃郁起司，包裹進蛋餅裡，療癒感十足。(此描述僅供參考),彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,75.0,2


In [10]:
def merge_csv_in_folder(folder_path):
    all_dfs = []
    col_name = ["shop_id", "shop_name", "review", "customer_score"]
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            df.columns = col_name
            all_dfs.append(df)

    # 使用 concat 進行垂直合併
    merged_df = pd.concat(all_dfs, ignore_index=True, axis=0)
    
    return merged_df

# 假設資料夾路徑是 'data/csv_files'
folder_path = './外送平台/GOOGLE評論/'
df_google_comment = merge_csv_in_folder(folder_path)

df_google_comment.head()


,shop_id,shop_name,review,customer_score
0,mq7f,早到晚到 (彰化中正店),彰化 新開 餐 飲店 火車 站 與員林客 運之間 一間 主打 晚餐 宵夜 店 彰化市 夜生活...,5 顆星
1,mq7f,早到晚到 (彰化中正店),東西 好吃 好吃 黑輪片 炸 黑 可怕,2 顆星
2,mq7f,早到晚到 (彰化中正店),其實 不用 賣 多品 項光 炒 泡 麵 稱霸 台灣 有點 缺點 炒泡 麵 建議 不要 淋巴肉...,4 顆星
3,mq7f,早到晚到 (彰化中正店),uber 上點 一下子 餐點 之後對 豬排 咖哩 飯 滿意 甜不辣 不錯 吃 法吐煉 乳覺 ...,5 顆星
4,mq7f,早到晚到 (彰化中正店),服務態度 餐點 不錯 可能 剛開幕 人手不足 等候 時間 很長,5 顆星


In [12]:
# 選擇 'shop_id' 和 'shop_name' 列，並移除重複的行
unique_shops = df_google_comment[["shop_id", "shop_name"]].drop_duplicates().reset_index(drop=True)

# unique_shops 現在包含了唯一的商店記錄
unique_shops.head()

,shop_id,shop_name
0,mq7f,早到晚到 (彰化中正店)
1,nz2a,茶經鮮奶茶專賣店
2,z9ey,一手私藏世界紅茶 (彰化和平店)
3,mp7q,春陽茶事 (彰化中正店)
4,e6ve,頂好馨臭臭鍋 (彰化店)


In [65]:
df_store = pd.merge(df_store, unique_shops, on="shop_name", how="left")

## 資料清整

In [15]:
df_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090625 entries, 0 to 1090624
Data columns (total 12 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   pic_id     1090625 non-null  object 
 1   food_name  1090625 non-null  object 
 2   food_id    1090625 non-null  int64  
 3   introduce  862505 non-null   object 
 4   address    1090625 non-null  object 
 5   longitude  1090625 non-null  float64
 6   latitude   1090625 non-null  float64
 7   label      1090625 non-null  object 
 8   shop_id    1090625 non-null  object 
 9   sort       760937 non-null   object 
 10  price      1090625 non-null  float64
 11  span       1090625 non-null  int64  
dtypes: float64(3), int64(2), object(7)
memory usage: 99.8+ MB


In [16]:
df_food['introduce'] = df_food['introduce'].fillna("no comment")

In [17]:
df_store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41281 entries, 0 to 41280
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   shop_name    41281 non-null  object
 1   url          41281 non-null  object
 2   total_score  41281 non-null  object
 3   food_type    41281 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [18]:
df_store['total_score'] = df_store['total_score'].apply(lambda x: x.split()[0])

In [20]:
df_google_comment['customer_score'] = df_google_comment['customer_score'].apply(lambda x: x.split()[0])

## 特徵工程

In [36]:
df_store

,shop_name,url,total_score,food_type
0,早到晚到 (彰化中正店),/restaurant/mq7f/zao-dao-wan-dao-zhang-hua-zho...,5,台式
1,火車頭小吃 (彰化店),/restaurant/pui4/huo-che-tou-xiao-chi-zhang-hu...,4.8,台式
2,可不可熟成紅茶 (彰化中正店),/restaurant/tdxz/ke-bu-ke-shou-cheng-hong-cha-...,4.9,飲料
3,霖生炒鴨肉羹,/restaurant/g58s/lin-sheng-chao-ya-rou-geng,4.5,台式
4,茶經鮮奶茶專賣店,/restaurant/nz2a/cha-jing-xian-nai-cha-zhuan-m...,4.9,飲料
...,...,...,...,...
41276,老味道便當小火鍋,/restaurant/hzli/lao-wei-dao-bian-dang-xiao-hu...,5,台式
41277,家文香雞排,/restaurant/t9bh/jia-wen-xiang-ji-pai,4.8,小吃
41278,胖達咖啡&茶專賣店 (麥寮忠孝店),/restaurant/t1yr/pang-da-ka-pei-andcha-zhuan-m...,4.9,歐美
41279,安木食堂,/restaurant/e9c9/an-mu-shi-tang,5,台式


In [37]:
df_store['common_shop_name'] = df_store['shop_name'].apply(lambda x: x.split()[0])

# 現在 df['common_shop_name'] 包含了不考慮分店的店名
df_store

,shop_name,url,total_score,food_type,common_shop_name
0,早到晚到 (彰化中正店),/restaurant/mq7f/zao-dao-wan-dao-zhang-hua-zho...,5,台式,早到晚到
1,火車頭小吃 (彰化店),/restaurant/pui4/huo-che-tou-xiao-chi-zhang-hu...,4.8,台式,火車頭小吃
2,可不可熟成紅茶 (彰化中正店),/restaurant/tdxz/ke-bu-ke-shou-cheng-hong-cha-...,4.9,飲料,可不可熟成紅茶
3,霖生炒鴨肉羹,/restaurant/g58s/lin-sheng-chao-ya-rou-geng,4.5,台式,霖生炒鴨肉羹
4,茶經鮮奶茶專賣店,/restaurant/nz2a/cha-jing-xian-nai-cha-zhuan-m...,4.9,飲料,茶經鮮奶茶專賣店
...,...,...,...,...,...
41276,老味道便當小火鍋,/restaurant/hzli/lao-wei-dao-bian-dang-xiao-hu...,5,台式,老味道便當小火鍋
41277,家文香雞排,/restaurant/t9bh/jia-wen-xiang-ji-pai,4.8,小吃,家文香雞排
41278,胖達咖啡&茶專賣店 (麥寮忠孝店),/restaurant/t1yr/pang-da-ka-pei-andcha-zhuan-m...,4.9,歐美,胖達咖啡&茶專賣店
41279,安木食堂,/restaurant/e9c9/an-mu-shi-tang,5,台式,安木食堂


In [59]:
cities_and_counties = [
    "臺北市", "新北市", "桃園市", "臺中市", "臺南市", "高雄市",
    "新竹縣", "苗栗縣", "彰化縣", "南投縣", "雲林縣", "嘉義縣",
    "屏東縣", "宜蘭縣", "花蓮縣", "臺東縣", "澎湖縣", "金門縣",
    "連江縣", "基隆市", "新竹市", "嘉義市", "彰化市"
]

cities_and_counties_english = [
    "Taipei City", "New Taipei City", "Taoyuan City", "Taichung City", "Tainan City", "Kaohsiung City",
    "Hsinchu County", "Miaoli County", "Changhua County", "Nantou County", "Yunlin County", "Chiayi County",
    "Pingtung County", "Yilan County", "Hualien County", "Taitung County", "Penghu County", "Kinmen County",
    "Lienchiang County", "Keelung City", "Hsinchu City", "Chiayi City", "Changhua City"
]

cities_and_counties_all = cities_and_counties + cities_and_counties_english

# 對DataFrame進行操作來添加新列 'county'
df_food['county'] = df_food['address'].apply(lambda address: next((county for county in cities_and_counties_all if county in address), ''))

# 顯示更新後的DataFrame
df_food.head()


,pic_id,food_name,food_id,introduce,address,longitude,latitude,label,shop_id,sort,price,span,county
0,mq7f_96743787.jpg,(F)手感豬肉漢堡,96743787,肉品原產地：台灣豬肉,彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,65.0,2,彰化縣
1,mq7f_96743788.jpg,(F)法式牛奶起司鍋燒麵,96743788,no comment,彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,188.0,4,彰化縣
2,mq7f_96743789.jpg,花生厚片,96743789,香濃滑順的花生醬搭配厚片，絲滑入口，等你帶我走！(此描述僅供參考),彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,65.0,2,彰化縣
3,mq7f_96743791.jpg,(F)濃厚乳酪花生法式吐司,96743791,香濃滑順的花生醬搭配吐司，絲滑入口，等你帶我走！(此描述僅供參考),彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,80.0,2,彰化縣
4,mq7f_96743792.jpg,薯泥起司蛋餅,96743792,將綿密薯泥、濃郁起司，包裹進蛋餅裡，療癒感十足。(此描述僅供參考),彰化縣彰化市中正路二段148號,24.078734,120.537428,Brunch,mq7f,服務,75.0,2,彰化縣


目的:
我想投資餐飲業，哪種餐廳最賺錢?

問題:
影響一間店的營收有哪些?


- 分店規模
- 客單價
- 評價
- 地點
- 餐點型態
- 品類數量

# 餐廳投資分析報告

## 目的
為了幫助投資者了解餐飲業的投資機會，本報告利用現有數據集進行深入分析，以確定哪些因素最影響餐廳的營收。

## 數據集
本報告基於以下數據集進行分析：
- `df_store`: 包含店鋪名稱、網址、總評分、餐點類型和店鋪ID。
- `df_meal`: 包括食品名稱、ID、介紹、地址、經緯度、標籤、店鋪ID、類別、價格和範圍。
- `df_google_comment`: 包含店鋪ID、店鋪名稱、顧客評論和客戶評分。

## 分析面向

### 1. 店鋪評價與營收分析
- 分析店鋪的總體評分與營業額之間的關聯性。
- 進行客戶評論的情感分析，了解其對營收的影響。

### 2. 餐點型態與客單價分析
- 探索不同餐點類型的平均價格與客單價的關係。
- 考慮菜單多樣性與店鋪績效之間的相關性。

### 3. 地理位置分析
- 利用經緯度數據分析地點對營業額的影響。

### 4. 綜合分析
- 結合市場趨勢分析，考慮不同餐廳風格的受歡迎程度。
- 分析評價與顧客流量的關聯。

### 5. 其他考慮因素
- 進行時段分析，了解不同時間的營業額變化。
- 考慮市場競爭對店鋪營收的影響。



### 衡量指標

- 平均單價: SUM(餐點價格 * 品項 ) / 總品項數量   
- 歡迎程度: 評論數量 * 平均星等 / 每一店家
- 粗估獲利能力: 平均單價 * 評論數量 * 分店數量

In [25]:
# 計算客單價
# 這裡假設 df_meal 中每行是一個品項

customer_price = df_food.groupby('shop_id')['price'].sum() / df_food.groupby('shop_id')['food_id'].count()

df_customer_price = pd.DataFrame(customer_price).reset_index()

df_customer_price.columns = ["shop_id", "avg_price"]

df_customer_price = df_customer_price.round(0)

In [26]:
df_customer_price

,shop_id,avg_price
0,a008,130.0
1,a02p,114.0
2,a03t,51.0
3,a06a,189.0
4,a08d,71.0
...,...,...
37694,zzrm,95.0
37695,zzsq,74.0
37696,zzuc,77.0
37697,zzvo,212.0


In [29]:
# 計算歡迎程度
# 轉換星級評分為數字
df_google_comment['numeric_score'] = df_google_comment['customer_score'].apply(lambda x: float(x.split()[0]))
popularity = (df_google_comment.groupby('shop_id')['review'].count() * df_google_comment.groupby('shop_id')['numeric_score'].mean())

df_popularity = pd.DataFrame(popularity).reset_index()

df_popularity.columns = ["shop_id", "hot score"]

df_popularity = df_popularity.round(0)

In [30]:
df_popularity

,shop_id,hot score
0,a008,144.0
1,a02p,424.0
2,a03t,440.0
3,a07o,170.0
4,a080,54.0
...,...,...
28190,zzrk,45.0
28191,zzrm,7.0
28192,zzsq,99.0
28193,zzvo,890.0


In [35]:
"""
有shop id 沒有shop name 代表沒有實體店，純走線上

"""

df_metrics = pd.merge(unique_shops, df_customer_price, on="shop_id", how="outer")

df_metrics = pd.merge(df_metrics, df_popularity, on="shop_id", how="outer")

df_metrics

,shop_id,shop_name,avg_price,hot score
0,mq7f,早到晚到 (彰化中正店),89.0,85.0
1,nz2a,茶經鮮奶茶專賣店,47.0,109.0
2,z9ey,一手私藏世界紅茶 (彰化和平店),64.0,281.0
3,mp7q,春陽茶事 (彰化中正店),67.0,219.0
4,e6ve,頂好馨臭臭鍋 (彰化店),167.0,304.0
...,...,...,...,...
39931,zzed,NaN,162.0,NaN
39932,zzn0,NaN,65.0,NaN
39933,zzr4,NaN,118.0,NaN
39934,zzuc,NaN,77.0,NaN


In [ ]:
# 計算獲利能力
# 假設 df_store 中有分店數量 'branch_count'
profitability = customer_price * df_google_comment.groupby('shop_id')['review'].count() * df_store['branch_count']

# 將計算結果整合到一個新的 DataFrame
final_df = pd.DataFrame({
    'Customer Price': customer_price,
    'Popularity': popularity,
    'Profitability': profitability
})

print(final_df)

In [89]:
df_store_main = df_store.groupby('common_shop_name')['common_shop_name'].count()

df_store_main = pd.DataFrame(df_store_main)

df_store_main.columns = ["branches_num"]

df_store_main.reset_index(inplace=True)

df_store_main

,common_shop_name,branches_num
0,!'M,1
1,&TEA,1
2,(AN)越南美食,1
3,(古根)土雞肉飯,1
4,(和)橋山豆漿大王,1
...,...,...
35363,ＢＬＵＥ營養三明治,1
35364,Ｗeilagirl微辣女孩,1
35365,���熹哞熹,1
35366,𡘙師傅便當專賣店,1


In [90]:
df_store

,shop_name,url,total_score,food_type,common_shop_name,shop_id
0,早到晚到 (彰化中正店),/restaurant/mq7f/zao-dao-wan-dao-zhang-hua-zho...,5,台式,早到晚到,mq7f
1,火車頭小吃 (彰化店),/restaurant/pui4/huo-che-tou-xiao-chi-zhang-hu...,4.8,台式,火車頭小吃,NaN
2,可不可熟成紅茶 (彰化中正店),/restaurant/tdxz/ke-bu-ke-shou-cheng-hong-cha-...,4.9,飲料,可不可熟成紅茶,NaN
3,霖生炒鴨肉羹,/restaurant/g58s/lin-sheng-chao-ya-rou-geng,4.5,台式,霖生炒鴨肉羹,NaN
4,茶經鮮奶茶專賣店,/restaurant/nz2a/cha-jing-xian-nai-cha-zhuan-m...,4.9,飲料,茶經鮮奶茶專賣店,nz2a
...,...,...,...,...,...,...
41384,老味道便當小火鍋,/restaurant/hzli/lao-wei-dao-bian-dang-xiao-hu...,5,台式,老味道便當小火鍋,NaN
41385,家文香雞排,/restaurant/t9bh/jia-wen-xiang-ji-pai,4.8,小吃,家文香雞排,t9bh
41386,胖達咖啡&茶專賣店 (麥寮忠孝店),/restaurant/t1yr/pang-da-ka-pei-andcha-zhuan-m...,4.9,歐美,胖達咖啡&茶專賣店,t1yr
41387,安木食堂,/restaurant/e9c9/an-mu-shi-tang,5,台式,安木食堂,e9c9


In [91]:
df_google_comment

,shop_id,shop_name,review,customer_score,numeric_score
0,mq7f,早到晚到 (彰化中正店),彰化 新開 餐 飲店 火車 站 與員林客 運之間 一間 主打 晚餐 宵夜 店 彰化市 夜生活...,5,5.0
1,mq7f,早到晚到 (彰化中正店),東西 好吃 好吃 黑輪片 炸 黑 可怕,2,2.0
2,mq7f,早到晚到 (彰化中正店),其實 不用 賣 多品 項光 炒 泡 麵 稱霸 台灣 有點 缺點 炒泡 麵 建議 不要 淋巴肉...,4,4.0
3,mq7f,早到晚到 (彰化中正店),uber 上點 一下子 餐點 之後對 豬排 咖哩 飯 滿意 甜不辣 不錯 吃 法吐煉 乳覺 ...,5,5.0
4,mq7f,早到晚到 (彰化中正店),服務態度 餐點 不錯 可能 剛開幕 人手不足 等候 時間 很長,5,5.0
...,...,...,...,...,...
3635938,wimv,Pizza Hut必勝客 (桃鶯鳳鳴外送店),好吃 服務 讚,5,5.0
3635939,wimv,Pizza Hut必勝客 (桃鶯鳳鳴外送店),服務好 服務人員 親切,5,5.0
3635940,wimv,Pizza Hut必勝客 (桃鶯鳳鳴外送店),好吃 皮烤 酥酥的,5,5.0
3635941,wimv,Pizza Hut必勝客 (桃鶯鳳鳴外送店),服務 迅速 滿意,5,5.0
